# Notebook 1: Loading snapshots with units and derived variables

This script shows how to

- load data: present in arepo snapshots and derived by Paicos
- how to convert the Arepo data from comoving code units to physical values in various unit systems


## Compilation
The first step is to compile the code, this only needs to be done the first time you use Paicos and if you have not already followed the installation instructions (replace the path to your own cloned version, you will also need to add this path to your PYTHONPATH).

In [1]:
#%%bash
#cd ~/projects/paicos
#make clean
#make

## Loading arepo snapshots

We load a zoom-in simulation of a galaxy cluster simulation below (the data is not included in the repo, so you will need to download this. See documentation)

In [2]:
import paicos as pa
import numpy as np

# A snapshot object
snap = pa.Snapshot(pa.data_dir, 247)

# The center of the most massive Friends-of-friends group in the simulation
center = snap.Cat.Group['GroupPos'][0]

## Useful metadata
We can look at some of snap attributes:

In [3]:
# Age of the Universe for the snapshot
snap.age

<PaicosQuantity 13.80223476 Gyr>

In [4]:
# The lookback time
snap.lookback_time

<PaicosQuantity 3.55271368e-15 Gyr>

In [5]:
# An astropy cosmology object (used internally to calculate the age and lookback, 
# cosmological parameters automatically loaded from the the snapshot)
snap.cosmo

LambdaCDM(H0=67.32117 km / (Mpc s), Om0=0.31582309, Ode0=0.68417691, Tcmb0=0.0 K, Neff=3.04, m_nu=None, Ob0=0.04938682)

In [6]:
# The size of the computational box
snap.box_size.to('Mpc').to_physical

<PaicosQuantity [1485.41704731, 1485.41704731, 1485.41704731] Mpc>

In [7]:
# The redshift
snap.z

2.220446049250313e-16

In [8]:
# Adiabatic index used in the simulation
snap.gamma

1.6666666666666667

In [9]:
# Contents of the three hdf5 groups containing information about the snapshot (uncomment to see output)
#snap.Header
#snap.Parameters
#snap.Config

## Loading of data blocks
Loading of data can be done using function calls or by trying to access them explicitly.
Here we load the Arepo data as a PaicosQuantity (basically a subclass of an astropy quantity), which gives the numeric data units and some useful methods. The numeric values are the same as stored in the hdf5 files but we can now see the units used in the simulation. Here small_h and small_a are the reduced Hubble parameter and the scale factor, respectively. 

In [10]:
# Load some variables from the PartType 0 (gas variables) 

# You can explicitly load using function call:
snap.load_data(0, 'Coordinates')
snap['0_Coordinates']

# But is much easier to just do it like this:
snap['0_Density']
snap['0_MagneticField']

# snap
snap['0_Volume']

Attempting to get derived variable: 0_Volume...	[DONE]



<PaicosQuantity [1.13548299e+03, 1.25744406e+03, 1.50093613e+03, ...,
                 8.87195460e+07, 6.21206842e+08, 7.11135763e+07] arepo_mass small_a3 / (arepo_density small_h3)>

In [11]:
# The available fields for a PartType can be found as shown below for parttype 0 (the gas)
keys = snap.info(0)

# alternatively, by starting to type and using tab-completion, i.e., snap['0_  and then hit tab


Keys for PartType0 in the hdf5 file:
0_AllowRefinement
0_CenterOfMass
0_Coordinates
0_Density
0_ElectronAbundance
0_EnergyDissipation
0_GFM_AGNRadiation
0_GFM_CoolingRate
0_GFM_Metallicity
0_GFM_Metals
0_GFM_WindDMVelDisp
0_GFM_WindHostHaloMass
0_HighResGasMass
0_InternalEnergy
0_Machnumber
0_MagneticField
0_MagneticFieldDivergence
0_MagneticFieldDivergenceAlternative
0_Masses
0_NeutralHydrogenAbundance
0_ParticleIDs
0_Potential
0_StarFormationRate
0_SubfindDMDensity
0_SubfindDensity
0_SubfindHsml
0_SubfindVelDisp
0_Velocities
0_VelocityGradient

Possible derived variables are:
0_Diameters
0_Enstrophy
0_EnstrophyTimesMasses
0_GFM_MetallicityTimesMasses
0_MachnumberTimesEnergyDissipation
0_MagneticFieldSquared
0_MagneticFieldSquaredTimesVolume
0_MagneticFieldStrength
0_MeanMolecularWeight
0_NumberDensity
0_Pressure
0_PressureTimesVolume
0_Temperatures
0_TemperaturesTimesMasses
0_VelocityCurvature
0_VelocityMagnitude
0_Volume


## Unit conversion 
Here we show how to use convert the density field to various useful physical units and
how to get rid of the a and h factors used in cosmological simulations with Arepo.

In [12]:
# Unit conversion
rho = snap['0_Density']
print('rho[0] in CGS:\t', rho[0].cgs)
print('rho[0] in SI:\t', rho[0].si)
print("rho[0] in 'astro' units:\t", rho[0].astro)
print("rho[0] in Msun/au^3:\t", rho[0].to('Msun/au3'), '\n\n')

# Get rid of h factors
print('rho[0] without h:\t', rho[0].no_small_h)

# Get rid of both a and h factors
print('rho[0] without a and h:\t', rho[0].to_physical, '\n\n')

# Get a label for use in plots
print(rho.label(r'\rho'))
print(rho.astro.label(r'\rho'))

rho[0] in CGS:	 6.586066423538464e-25 g small_h2 / (cm3 small_a3)
rho[0] in SI:	 6.586066423538463e-22 kg small_h2 / (m3 small_a3)
rho[0] in 'astro' units:	 9731321.74428584 small_h2 solMass / (kpc3 small_a3)
rho[0] in Msun/au^3:	 1.1089103378991942e-18 small_h2 solMass / (AU3 small_a3) 


rho[0] without h:	 0.00044090644374425626 arepo_density / small_a3
rho[0] without a and h:	 0.00044090644374425653 arepo_density 


$\rho\;a^{-3}h^{2}\; \left[\mathrm{arepo\_density}\right]$
$\rho\;a^{-3}h^{2}\; \left[\mathrm{M_{\odot}}\;\mathrm{kpc}^{-3}\right]$


### Changing the units in a PaicosQuantity
Please note that the methods above return a new object without modifying the original data. Modification can be done by overwriting, e.g., like this:

In [13]:
rho = rho.to_physical
rho

<PaicosQuantity [4.40906444e-04, 4.11126523e-04, 3.81600963e-04, ...,
                 9.60358397e-09, 3.67888283e-10, 1.18164277e-08] arepo_density>

### Getting rid of units

The PaicosQuantity uses the astropy quantity internally, which again uses numpy arrays.
In case you are not familiar with astropy: Here is how you can access the unit and numeric values independently like this: 

In [14]:
# The unit
rho.unit

Unit("arepo_density")

In [15]:
# The numeric values (a numpy array)
rho.value

array([4.40906444e-04, 4.11126523e-04, 3.81600963e-04, ...,
       9.60358397e-09, 3.67888283e-10, 1.18164277e-08])

## Loading automatically computed derived variables
Here we show how Paicos can automatically compute derived variables.
Paicos gives information about what is happening under the hood

This feature can be turned off by setting

```
pa.settings.use_only_user_functions = True
```
but we note that changes to `pa.settings.use_only_user_functions` only take effect for freshly loaded snapshots.

In [16]:
# The gas cell volumes (per default Arepo only outputs the gas mass and density)
snap['0_Volume']

# The gas temperature times the cell gas mass
snap['0_TemperaturesTimesMasses']

# The gas enstrophy
snap['0_Enstrophy']

Attempting to get derived variable: 0_TemperaturesTimesMasses...
	So we need the variable: 0_Temperatures...
	So we need the variable: 0_MeanMolecularWeight...	[DONE]

Attempting to get derived variable: 0_Enstrophy...	[DONE]



<PaicosQuantity [2.05483272e+03, 3.71073260e+03, 4.08898476e+03, ...,
                 4.45839155e-01, 1.79965047e-04, 7.33761679e-02] arepo_velocity2 small_h2 / (arepo_length2 small_a)>

The console output can be turned off by setting
```
pa.settings.print_info_when_deriving_variables = False
```
This is illustrated below where no information is printed:

In [17]:
# Turn off info
pa.settings.print_info_when_deriving_variables = False

# The metallicity multiplied by the mass
snap['0_GFM_MetallicityTimesMasses']

# Turn info back on
pa.settings.print_info_when_deriving_variables = True